In [143]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [144]:
import os
import re
# import xml.etree.ElementTree as ET

import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [148]:
"""
n02058221 albatross, mollymawk
n02130308 cheetah, chetah, Acinonyx jubatus

n01518878 ostrich, Struthio camelus
n02056570 king penguin, Aptenodytes patagonica
# n02123159 tiger cat

"""


mapping_dict = {
    'n02058221': 'albatross',
    'n02130308': 'cheetah',
    'n01518878': 'ostrich',
    'n02056570': 'penguin',
    'n02391049': 'zebra'
}


def extract_unique_color(img, rgb=False, normalize=False):
    row, col, _ = img.shape
    reshaped_img = img.reshape(row * col, 3)
    unique_color_array = np.unique(reshaped_img, axis=0)

    if rgb:
        # rgbに要素を並び替え
        unique_color_array = unique_color_array[:, [2, 1, 0]]

    if normalize:
        unique_color_array = unique_color_array / 255

    return unique_color_array

In [149]:
print(mapping_dict.values())

dict_values(['albatross', 'cheetah', 'ostrich', 'penguin', 'zebra'])


In [150]:
work_dir_path = './imagenet/ILSVRC/Data/CLS-LOC/train/'
target_dir_names = os.listdir(work_dir_path)
target_dir_names

['n02056570', 'n01518878', 'n02058221', 'n02130308', 'n02391049']

In [151]:
for target_dir_name in target_dir_names:
    print(len(os.listdir(os.path.join(work_dir_path, target_dir_name))))

1300
1300
1300
1300
1300


In [49]:
path = os.path.join(work_dir_path, target_dir_names[0])
file_names_n02056570 = os.listdir(path)
file_names_n02056570

['n02056570_8065.JPEG',
 'n02056570_8610.JPEG',
 'n02056570_8445.JPEG',
 'n02056570_1188.JPEG',
 'n02056570_8237.JPEG',
 'n02056570_10362.JPEG',
 'n02056570_9030.JPEG',
 'n02056570_4836.JPEG',
 'n02056570_1970.JPEG',
 'n02056570_8242.JPEG',
 'n02056570_333.JPEG',
 'n02056570_8295.JPEG',
 'n02056570_1924.JPEG',
 'n02056570_9785.JPEG',
 'n02056570_8684.JPEG',
 'n02056570_1691.JPEG',
 'n02056570_9638.JPEG',
 'n02056570_9227.JPEG',
 'n02056570_9464.JPEG',
 'n02056570_7339.JPEG',
 'n02056570_4215.JPEG',
 'n02056570_8369.JPEG',
 'n02056570_9041.JPEG',
 'n02056570_15798.JPEG',
 'n02056570_3318.JPEG',
 'n02056570_8797.JPEG',
 'n02056570_936.JPEG',
 'n02056570_9720.JPEG',
 'n02056570_9666.JPEG',
 'n02056570_6235.JPEG',
 'n02056570_9303.JPEG',
 'n02056570_9232.JPEG',
 'n02056570_9628.JPEG',
 'n02056570_9615.JPEG',
 'n02056570_1608.JPEG',
 'n02056570_12752.JPEG',
 'n02056570_863.JPEG',
 'n02056570_2175.JPEG',
 'n02056570_13462.JPEG',
 'n02056570_9302.JPEG',
 'n02056570_1396.JPEG',
 'n02056570_724

In [55]:
file_name = os.path.join(work_dir_path, target_dir_names[0], 'n02056570_8065.JPEG')
cv2.imread(file_name).shape

(1200, 1600, 3)

In [52]:
file_name = os.path.join(work_dir_path, target_dir_names[0], 'n02056570_8065.JPEG')

'./imagenet/ILSVRC/Data/CLS-LOC/train/n02056570/n02056570_8065.JPEG'

In [67]:
os.path.join(work_dir_path, target_dir_names[0])

'./imagenet/ILSVRC/Data/CLS-LOC/train/n02056570'

In [68]:
os.listdir(os.path.join(work_dir_path, target_dir_names[0]))

['n02056570_8065.JPEG',
 'n02056570_8610.JPEG',
 'n02056570_8445.JPEG',
 'n02056570_1188.JPEG',
 'n02056570_8237.JPEG',
 'n02056570_10362.JPEG',
 'n02056570_9030.JPEG',
 'n02056570_4836.JPEG',
 'n02056570_1970.JPEG',
 'n02056570_8242.JPEG',
 'n02056570_333.JPEG',
 'n02056570_8295.JPEG',
 'n02056570_1924.JPEG',
 'n02056570_9785.JPEG',
 'n02056570_8684.JPEG',
 'n02056570_1691.JPEG',
 'n02056570_9638.JPEG',
 'n02056570_9227.JPEG',
 'n02056570_9464.JPEG',
 'n02056570_7339.JPEG',
 'n02056570_4215.JPEG',
 'n02056570_8369.JPEG',
 'n02056570_9041.JPEG',
 'n02056570_15798.JPEG',
 'n02056570_3318.JPEG',
 'n02056570_8797.JPEG',
 'n02056570_936.JPEG',
 'n02056570_9720.JPEG',
 'n02056570_9666.JPEG',
 'n02056570_6235.JPEG',
 'n02056570_9303.JPEG',
 'n02056570_9232.JPEG',
 'n02056570_9628.JPEG',
 'n02056570_9615.JPEG',
 'n02056570_1608.JPEG',
 'n02056570_12752.JPEG',
 'n02056570_863.JPEG',
 'n02056570_2175.JPEG',
 'n02056570_13462.JPEG',
 'n02056570_9302.JPEG',
 'n02056570_1396.JPEG',
 'n02056570_724

In [70]:
work_dir_path

'./imagenet/ILSVRC/Data/CLS-LOC/train/'

In [71]:
os.path.join(work_dir_path, 'n02130308')

'./imagenet/ILSVRC/Data/CLS-LOC/train/n02130308'

In [72]:
os.listdir(os.path.join(work_dir_path, 'n02130308'))

['n02130308_16613.JPEG',
 'n02130308_9189.JPEG',
 'n02130308_16307.JPEG',
 'n02130308_21788.JPEG',
 'n02130308_17105.JPEG',
 'n02130308_4844.JPEG',
 'n02130308_22849.JPEG',
 'n02130308_23917.JPEG',
 'n02130308_18264.JPEG',
 'n02130308_6281.JPEG',
 'n02130308_21393.JPEG',
 'n02130308_21267.JPEG',
 'n02130308_10550.JPEG',
 'n02130308_16807.JPEG',
 'n02130308_3210.JPEG',
 'n02130308_1669.JPEG',
 'n02130308_348.JPEG',
 'n02130308_6197.JPEG',
 'n02130308_11649.JPEG',
 'n02130308_2425.JPEG',
 'n02130308_9291.JPEG',
 'n02130308_2899.JPEG',
 'n02130308_10233.JPEG',
 'n02130308_9983.JPEG',
 'n02130308_15584.JPEG',
 'n02130308_6698.JPEG',
 'n02130308_19927.JPEG',
 'n02130308_8457.JPEG',
 'n02130308_326.JPEG',
 'n02130308_549.JPEG',
 'n02130308_857.JPEG',
 'n02130308_2478.JPEG',
 'n02130308_1888.JPEG',
 'n02130308_20288.JPEG',
 'n02130308_1708.JPEG',
 'n02130308_8582.JPEG',
 'n02130308_1869.JPEG',
 'n02130308_1484.JPEG',
 'n02130308_10838.JPEG',
 'n02130308_23960.JPEG',
 'n02130308_431.JPEG',
 'n

In [152]:
data_list = []

for dir_name in target_dir_names:
    target_dir_path = os.path.join(work_dir_path, dir_name)
    file_names = os.listdir(target_dir_path)
    
    for file_name in tqdm(file_names, desc=f"Processing {dir_name}"):
        file_path = os.path.join(target_dir_path, file_name)
        img = cv2.imread(file_path)
        width, height, depth = img.shape

        arr_rgb = extract_unique_color(img, rgb=True, normalize=True)
        arr_rgb_means = arr_rgb.mean(0)
        r, g, b = arr_rgb_means[0], arr_rgb_means[1], arr_rgb_means[2]

        data = {
            'folder': dir_name,
            'name': re.search(r'_(.*?)\.', file_name).group(1),
            'width': width,
            'height': height,
            'depth': depth,
            'R': r,
            'G': g,
            'B': b
        }

        data_list.append(data)


Processing n02056570:   0%|          | 0/1300 [00:00<?, ?it/s]

Processing n02056570:  74%|███████▎  | 956/1300 [06:45<02:26,  2.36it/s]  


KeyboardInterrupt: 

In [153]:
data_list = []

dir_name = 'n02391049'
target_dir_path = os.path.join(work_dir_path, dir_name)
file_names = os.listdir(target_dir_path)

for file_name in tqdm(file_names, desc=f"Processing {dir_name}"):
    file_path = os.path.join(target_dir_path, file_name)
    img = cv2.imread(file_path)
    width, height, depth = img.shape

    arr_rgb = extract_unique_color(img, rgb=True, normalize=True)
    arr_rgb_means = arr_rgb.mean(0)
    r, g, b = arr_rgb_means[0], arr_rgb_means[1], arr_rgb_means[2]

    data = {
        'folder': dir_name,
        'name': re.search(r'_(.*?)\.', file_name).group(1),
        'width': width,
        'height': height,
        'depth': depth,
        'R': r,
        'G': g,
        'B': b
    }

    data_list.append(data)


Processing n02391049: 100%|██████████| 1300/1300 [06:34<00:00,  3.29it/s]


In [154]:
df = pd.DataFrame(data_list)
df.to_csv('stats_additional.csv')

display(df)
display(df.describe())

,folder,name,width,height,depth,R,G,B
0,n02391049,3221,333,500,3,0.488657,0.452150,0.378734
1,n02391049,22,360,480,3,0.469938,0.527937,0.361228
2,n02391049,890,600,800,3,0.524513,0.543636,0.491845
3,n02391049,2363,284,427,3,0.500000,0.500000,0.500000
4,n02391049,19393,310,500,3,0.562834,0.515805,0.399424
...,...,...,...,...,...,...,...,...
1295,n02391049,2311,333,500,3,0.618658,0.583356,0.521907
1296,n02391049,6650,379,500,3,0.506803,0.401962,0.288625
1297,n02391049,2627,500,375,3,0.517260,0.509384,0.498286
1298,n02391049,7258,375,500,3,0.470308,0.439706,0.373938


,width,height,depth,R,G,B
count,1300.000000,1300.000000,1300.0,1300.000000,1300.000000,1300.000000
mean,407.856154,500.535385,3.0,0.500638,0.478321,0.400873
std,123.875655,135.728829,0.0,0.065261,0.055927,0.077209
min,104.000000,120.000000,3.0,0.277202,0.270841,0.096994
25%,333.000000,500.000000,3.0,0.463765,0.445262,0.350629
50%,375.000000,500.000000,3.0,0.500000,0.480280,0.402201
75%,499.250000,500.000000,3.0,0.540598,0.511125,0.453539
max,2048.000000,2560.000000,3.0,0.803800,0.679188,0.762021


In [126]:
df_data = df.loc[:, ["R", "G", "B"]]
df_labels = df.loc[:, "folder"]

display(df_data.head())
display(df_labels.head())

,R,G,B
0,0.486640,0.440085,0.342231
1,0.427333,0.406967,0.376660
2,0.509321,0.512104,0.398580
3,0.581645,0.572609,0.549550
4,0.505475,0.517679,0.422280


0    n02056570
1    n02056570
2    n02056570
3    n02056570
4    n02056570
Name: folder, dtype: object

In [133]:
data = np.array(df_data)
labels = np.array(df_labels)

In [128]:
arr_data

array([[0.48664018, 0.44008494, 0.3422309 ],
       [0.42733264, 0.4069667 , 0.37666028],
       [0.50932126, 0.512104  , 0.39857979],
       ...,
       [0.49182425, 0.53119295, 0.46209472],
       [0.46971513, 0.48360581, 0.37751026],
       [0.48814955, 0.45962042, 0.28293   ]])

In [131]:
np.unique(arr_labels)

array(['n01518878', 'n02056570', 'n02058221', 'n02130308'], dtype=object)

In [134]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report


In [135]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

In [140]:
svm_model = SVC(kernel='linear', C=1.0)
svm_model.fit(X_train, y_train)

SVC(kernel='linear')

In [141]:
y_pred = svm_model.predict(X_test)

# 分類精度を評価
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# クラスごとの詳細な評価を表示
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)


Accuracy: 0.4480769230769231
Classification Report:
               precision    recall  f1-score   support

   n01518878       0.38      0.38      0.38       377
   n02056570       0.38      0.32      0.34       409
   n02058221       0.53      0.62      0.57       366
   n02130308       0.49      0.49      0.49       408

    accuracy                           0.45      1560
   macro avg       0.44      0.45      0.45      1560
weighted avg       0.44      0.45      0.44      1560

